In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import ipywidgets as widgets
from pprint import pprint
import json

In [2]:
# Import API key
from config import weather_api_key
from config import g_key
gmaps.configure(api_key=g_key)

# Store Part 1 Results into DF

In [3]:
#Store Part I results into DataFrame by loading the csv exported in Part I to a DataFrame
vacay_df = pd.read_csv("../output_data/cities.csv")
#vacay_df["Hotel Name"]=""
vacay_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,qaanaaq,77.4840,-69.3632,28.60,74,61,4.45,GL,2021-04-25 14:15:23
1,lavrentiya,65.5833,-171.0000,21.07,88,100,3.91,RU,2021-04-25 14:12:15
2,koumac,-20.5667,164.2833,75.15,85,100,11.52,NC,2021-04-25 14:15:23
3,buraydah,26.3260,43.9750,98.60,13,75,10.36,SA,2021-04-25 14:15:24
4,mahebourg,-20.4081,57.7000,78.80,78,75,13.80,MU,2021-04-25 14:15:24


# Humidity Heatmap

In [4]:
locations = vacay_df[["Lat","Lng"]]
weights = vacay_df["Humidity"]

In [5]:
#Humidity Heatmap THAT DISPLAYS HUMIDITY FOR EVERY CITY FROM PART 1
#Use the Lat and Lng as locations and Humidity as the weight.
fig1 = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights, dissipating=False, max_intensity=150, point_radius = 4)
fig1.add_layer(heat_layer)
plt.savefig("../output_data/Map1")
fig1

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

# Create New DF Fitting Weather Criteria

In [6]:
#Create new DataFrame fitting weather criteria
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.
#MAx temp < 80 but > 70, wind speed < 10 mph, zero cloudiness - DROP ANY ROWS THAT DON'T MEET ALL 3 CONDITIONS 
ideal_df = vacay_df.loc[(vacay_df["Max Temp"] <77) & (vacay_df["Max Temp"] >70) & (vacay_df["Wind Speed"] <10) & (vacay_df["Cloudiness"] ==0)]
#ideal_df.describe()
ideal_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
129,east london,-33.0153,27.9116,71.60,68,0,6.91,ZA,2021-04-25 14:15:54
217,richards bay,-28.7830,32.0377,74.55,67,0,7.05,ZA,2021-04-25 14:16:16
311,guaxupe,-21.3053,-46.7128,73.40,49,0,2.75,BR,2021-04-25 14:16:40
400,machado,-21.6747,-45.9197,73.40,49,0,1.88,BR,2021-04-25 14:17:02


# Hotel Map

In [7]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Store into variable named hotel_df.
#"TEN OR LESS PINS FOR ALL CITIES IN THE DF"
# "PINS ARE ALL CLICKABLE TO DISPLAY CITY, COUNTRY, AND HOTEL NAME"

In [8]:
Hlat = []
Hlon = []
Hcountry = []
Hcity=[]
Hname = []
Haddress = []

In [9]:
for index, row in ideal_df.iterrows():
    try:
        lat = row["Lat"]
        lng = row["Lng"]
        target_coordinates =  f"{lat},{lng}"
        target_search = "hotel"
        target_radius = 6500
        #target_type = "hotel"
        
        params = {
            "location": target_coordinates,
            "keyword": target_search,
            "radius": target_radius,
            #"type": target_type,
            "key": g_key
            }   
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        names_etc = requests.get(base_url, params=params).json()
        #names_etc = name_etc.json()
        #pprint(names_etc)
        Hname.append(names_etc["results"][0]["name"])
        Hlat.append(names_etc["results"][0]["geometry"]["location"]["lat"])
        Hlon.append(names_etc["results"][0]["geometry"]["location"]["lng"])
        Hcountry.append(row["Country"])
        Hcity.append(row["City"])
        print(f"The hotel {names_etc['results'][0]['name']} was found with the lat of {names_etc['results'][0]['geometry']['location']['lat']} and lon of {names_etc['results'][0]['geometry']['location']['lng']}, appending stats.")
    except KeyError:
        print("Sorry, that hotel was not found.")
        pass

The hotel Blue Lagoon Hotel was found with the lat of -32.9834715 and lon of 27.9510976, appending stats.
The hotel BON Hotel Waterfront Richards Bay was found with the lat of -28.7932974 and lon of 32.0807935, appending stats.
The hotel Class Hotel Guaxupé was found with the lat of -21.3102498 and lon of -46.7085795, appending stats.
The hotel Marine Inn Hotel was found with the lat of -21.675579 and lon of -45.921833, appending stats.


In [10]:
hotel_df = pd.DataFrame({"Hotel Name": Hname, 
                       "Hotel Lat" : Hlat,
                       "Hotel Lng" : Hlon,
                       "Hotel Country": Hcountry,
                       "Hotel City" : Hcity
                        })
#all_df["Date"] = pd.to_datetime(all_df["Date"], unit ='s')
hotel_df.head(20)

,Hotel Name,Hotel Lat,Hotel Lng,Hotel Country,Hotel City
0,Blue Lagoon Hotel,-32.983472,27.951098,ZA,east london
1,BON Hotel Waterfront Richards Bay,-28.793297,32.080793,ZA,richards bay
2,Class Hotel Guaxupé,-21.310250,-46.708579,BR,guaxupe
3,Marine Inn Hotel,-21.675579,-45.921833,BR,machado


In [11]:
############ NOTE: Do not change any of the code in this cell###############
####### THIS IS HTML TEMPLATE################

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Hotel City}</dd>
<dt>Country</dt><dd>{Hotel Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[['Hotel Lat', 'Hotel Lng']]
#hotel_info

In [12]:
# "PINS ARE ALL CLICKABLE TO DISPLAY CITY, COUNTRY, AND HOTEL NAME"
fig1

#fig1 = gmaps.figure()
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig1.add_layer(markers)
fig1

Figure(layout=FigureLayout(height='420px'))